In [ ]:
import src.basic

main path: /Users/mengkjin/workspace/learndl
src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']


In [ ]:
from src.data.download.other_source.rcquant import RcquantMinBarDownloader
# RcquantMinBarDownloader.proceed()


In [ ]:
import pandas as pd
import numpy as np

from src.data.download.other_source.rcquant.bar_min import rqdatac


from typing import Literal
from datetime import datetime , timedelta
from src.basic import PATH
from src.func import date_diff , today

START_DATE = 20241101
RC_PATH = PATH.data.joinpath('Rcquant')

final_path = RC_PATH.joinpath(f'1min')
secdf_path = RC_PATH.joinpath('secdf')

final_path.mkdir(exist_ok=True , parents=True)
secdf_path.mkdir(exist_ok=True , parents=True)

def date_offset(date : int , n = 1):
    return int((datetime.strptime(str(date), '%Y%m%d') + timedelta(days=n)).strftime('%Y%m%d'))

def rcquant_secdf(date : int):
    path = secdf_path.joinpath(f'secdf_{date}.feather')
    if path.exists():
        return pd.read_feather(path)
    
    secdf = rqdatac.all_instruments(type='CS', date=str(date))
    secdf = secdf.rename(columns = {'order_book_id':'code'})
    secdf['is_active'] = secdf['status'] == 'Active'
    secdf.to_feather(path)
    return secdf

def rcquant_past_dates(file_type : Literal['secdf' , 'min']):
    path = final_path if file_type == 'min' else secdf_path
    past_files = [p for p in path.iterdir()]
    past_dates = sorted([int(p.name.split('.')[-2][-8:]) for p in past_files])
    return past_dates
    
def updated_dates():
    return rcquant_past_dates('min')

def last_date(offset : int = 0):
    dates = updated_dates()
    last_dt = max(dates) if len(dates) > 0 else date_offset(START_DATE , -1)
    return date_offset(last_dt , offset)

def trading_dates(start_date, end_date):
    return [int(td.strftime('%Y%m%d')) for td in  rqdatac.get_trading_dates(start_date, end_date, market='cn')]

def rquant_proceed(date : int | None = None , first_n : int = -1 , retry_n : int = 10):
    start_date = last_date(1)
    end_date = today(-1) if date is None else date
    for dt in trading_dates(start_date , end_date):
        mark = rquant_bar_min(dt , first_n)
        if not mark: 
            print(f'rcquant bar min {dt} failed')
            return False
        else:
            print(f'rcquant bar min {dt} success')
    return True

def rquant_bar_min(date : int , first_n : int = -1):
    def code_map(x : str):
        x = x.split('.')[0]
        if x[:1] in ['3', '0']:
            y = x+'.SZ'
        elif x[:1] in ['6']:
            y = x+'.SH'
        else:
            y = x
        return y

    # date = 20240704
    sec_df = rcquant_secdf(date)
    sec_df = sec_df[sec_df['is_active']]
    if first_n > 0: sec_df = sec_df.iloc[:first_n]
    stock_list = sec_df['code'].to_numpy()
    data = rqdatac.get_price(stock_list, start_date=date, end_date=date, frequency='1m',expect_df=True)
    if isinstance(data , pd.DataFrame):
        data = data.reset_index().rename(columns = {'total_turnover':'amount', 'order_book_id':'code'}).assign(date = date)
        data['code'] = data['code'].map(code_map)
        data['time'] = data['datetime'].map(lambda x: x.strftime('%H%M%S')).str.slice(0,4)
        data['date'] = data['datetime'].map(lambda x: x.strftime('%Y%m%d'))

        data.to_feather(final_path.joinpath(f'min_bar_{date}.feather'))

        df = rcquant_min_to_normal_min(data)
        PATH.db_save(df , 'trade_ts' , 'min' , date = date)
        return True
    else:
        return False

def rcquant_min_to_normal_min(df : pd.DataFrame):
    df = df.copy()
    df.loc[:,['open','high','low','close','volume','amount']] = df.loc[:,['open','high','low','close','volume','amount']].astype(float)
    df['secid'] = df['code'].str.extract(r'\.(\w+)$').astype(int)
    df['minute'] = ((df['time'].str.slice(8,10).astype(int) - 9.5) * 12 + df['time'].str.slice(10,12).astype(int) / 5).astype(int) - 1
    df.loc[df['minute'] >= 24 , 'minute'] -= 18
    df['vwap'] = df['amount'] / df['volume'].where(df['volume'] > 0 , np.nan)
    df['vwap'] = df['vwap'].where(df['vwap'].notna() , df['open'])
    df = df.loc[:,['secid','minute','open','high','low','close','amount','volume','vwap']].sort_values(['secid','minute']).reset_index(drop = True)
    return df

In [ ]:
import rqdatac
import pandas as pd
import numpy as np

from src.data.download.other_source.rcquant.bar_min import rcquant_settings
uri = rcquant_settings()['uri']
rqdatac.init(uri = uri)
date = '20241106'
def code_map(x):
    x = x.split('.')[0]
    if x[:1] in ['3', '0']:
        y = x+'.SZ'
    elif x[:1] in ['6']:
        y = x+'.SH'
    else:
        y = x
    return y

def rcquant_min_to_normal_min(df : pd.DataFrame):
    df = df.copy()
    df.loc[:,['open','high','low','close','volume','amount']] = df.loc[:,['open','high','low','close','volume','amount']].astype(float)
    df['secid'] = df['code'].str.extract(r'^(\w+)\.').astype(int)
    df['minute'] = ((df['time'].str.slice(8,10).astype(int) - 9.5) * 12 + df['time'].str.slice(10,12).astype(int) / 5).astype(int) - 1
    df.loc[df['minute'] >= 24 , 'minute'] -= 18
    df['vwap'] = df['amount'] / df['volume'].where(df['volume'] > 0 , np.nan)
    df['vwap'] = df['vwap'].where(df['vwap'].notna() , df['open'])
    df = df.loc[:,['secid','minute','open','high','low','close','amount','volume','vwap']].sort_values(['secid','minute']).reset_index(drop = True)
    return df

stock_list = rqdatac.all_instruments(type='CS', date=date)
stock_list = sorted(set(stock_list.loc[stock_list['status']=='Active', 'order_book_id']))


c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\rqdatac\client.py:257: UserWarning: Your account will be expired after  27 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [13]:
data = rqdatac.get_price(stock_list, start_date=date, end_date=date, frequency='1m',expect_df=True)
if data is not None:
    data = data.reset_index()
    cols_map = {'total_turnover':'amount', 'order_book_id':'code'}
    data.rename(columns = cols_map , inplace = True)
    data['code'] = data['code'].map(code_map)
    data['time'] = data['datetime'].map(lambda x: x.strftime('%H%M%S')).str.slice(0,4)
    data['date'] = data['datetime'].map(lambda x: x.strftime('%Y%m%d'))
    df = rcquant_min_to_normal_min(data)

data

,code,datetime,amount,close,high,open,num_trades,volume,low,time,date
0,000001.SZ,2024-11-06 09:31:00,67813102.0,11.59,11.62,11.62,1719.0,5843900.0,11.58,0931,20241106
1,000001.SZ,2024-11-06 09:32:00,21205760.0,11.58,11.60,11.60,811.0,1831500.0,11.57,0932,20241106
2,000001.SZ,2024-11-06 09:33:00,29478407.0,11.60,11.61,11.58,658.0,2543610.0,11.58,0933,20241106
3,000001.SZ,2024-11-06 09:34:00,15537465.0,11.60,11.61,11.60,543.0,1339800.0,11.59,0934,20241106
4,000001.SZ,2024-11-06 09:35:00,17002986.0,11.59,11.60,11.60,431.0,1466400.0,11.59,0935,20241106
...,...,...,...,...,...,...,...,...,...,...,...
2395,000012.SZ,2024-11-06 14:56:00,2808490.0,5.59,5.59,5.59,138.0,503100.0,5.58,1456,20241106
2396,000012.SZ,2024-11-06 14:57:00,1042369.0,5.59,5.59,5.58,165.0,186600.0,5.58,1457,20241106
2397,000012.SZ,2024-11-06 14:58:00,16186.0,5.58,5.58,5.58,5.0,2900.0,5.58,1458,20241106
2398,000012.SZ,2024-11-06 14:59:00,0.0,5.58,5.58,5.58,0.0,0.0,5.58,1459,20241106
